In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import polars as pl
import duckdb
import os
import pandera as pa
import io

FROM_EXPERIMENT_CSV_IN = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_02_EXP_SELECT_CIK_CSV")
TO_EXPERIMENT_PARQUET = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_03_EXP_SELECT_CIK_PARQUET")

In [3]:
%%time
# option 1: loading files with os.walk

columns = ['cik', 'cusip8', 'cusip9','rdate', 'fdate','value', 'shares',\
           'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type']

# columns2 = ['Address', 'Cik', 'Fdate', 'Rdate','Form', 'Value', 'Shares', 'Cusip8', \
#            'Cusip9', 'Shrs Or Prn Amt', 'Put Call', 'Name Of Issuer', 'Title Of Class', 'type']

dtypes = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': str,
          'putCall': str, 'shrsOrPrnAmt': str, 'value': float, 'shares': float, 
          'nameOfIssuer': str, 'cik' : int, 'address': str, 'type': str,'num5': str,
          'deviation':str, 'shrout':str,'num3': str,'num2': str, 'num6':str,'num7': str,'num4': str,
           'votingAuthority': str, 'in_universe': str,'prc': str, 'split': str,
           'investmentDiscretion': str}
    
# if df['fdate'].max() >= pd.to_datetime('2014-01-01',format='%Y-%m-%d') : continue

# def concatenate_dataframes_pd_pathlib(dir_path):
#     big_df = pd.DataFrame()
#     for file in dir_path.rglob("*.csv"):
#         df = pd.read_csv(file, usecols=lambda c: c in set(columns), parse_dates=['rdate','fdate'], dtype=dtypes)
#         big_df = pd.concat([big_df, df])
#     return big_df

def concatenate_dataframes_pl_pathlib(dir_path):
    big_df = pl.DataFrame()
    for file in dir_path.rglob("*.csv"):
        # print(file)
        df = pl.read_csv(file, parse_dates=True, dtypes=dtypes ) #[columns]
        # print(df.columns)
        # print(file, df.head, df.columns, sep="\n")
        for col in df.columns:
    # if the column does not exist in the list, drop it from the dataframe
            if col not in columns:
                df = df.drop(columns=[col])
        for col in columns:
            if col not in df.columns:
                df = df.with_column(pl.lit(None, dtype=dtypes[col]).alias(col))
        df = df.select(columns)
               

                
        # print(df.columns)
        # print(df.columns)
        try:
            big_df = pl.concat([big_df, df])
            # print(df.head, df.columns)
        except:
            print(df.head, df.columns)
    return big_df


CPU times: total: 0 ns
Wall time: 0 ns


In [ ]:
%%time
pl_df = concatenate_dataframes_pl_pathlib(FROM_EXPERIMENT_CSV_IN)

In [18]:
pl_df.head()

shape: (0, 0)
┌┐
╞╡
└┘

In [54]:
all_cols = ['cusip8', 'cusip9', 'shrsOrPrnAmt', 'putCall', 'investmentDiscretion', 'remainder', 'value', 'shares', 'permno', 'prc', 'shrout', 'split', 'deviation', 'address', 'rdate', 'fdate', 'cik', 'form', 'type']
columns = ['cik', 'cusip8', 'cusip9','rdate', 'fdate','value', 'shares',\
           'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type']

l12 = ['cusip8', 'cusip9', 'shrsOrPrnAmt', 'putCall', 'value', 'shares', 'address', 'rdate', 'fdate', 'cik', 'form', 'type', 'nameOfIssuer', 'titleOfClass']
l13 = ['cusip8', 'nameOfIssuer', 'titleOfClass', 'value', 'shrsOrPrnAmt', 'cusip9', 'shares', 'address', 'rdate', 'fdate', 'cik', 'form', 'type', 'putCall']

len(columns)
# np.setdiff1d(l12,columns)

14

In [8]:
txt = """
AB,CD,EF, JJ
foo,20160101,a,23
foo,20160102,a,34
foo,20160103,a,56
"""

data = pl.read_csv(io.StringIO(txt))
data = data.

[Utf8, Int64, Utf8, Int64]

In [ ]:
%%time

dfs = []
for file in FROM_EXPERIMENT_CSV_IN.rglob("*.csv"):
    print(file)
    schema = pl.scan_csv(file).schema
    
    read_cols = list(set(schema.keys()).intersection(columns))
    
    df = pl.read_csv(file, columns=read_cols, dtypes=dtypes)
    # if our `df` misses a column from the `columns` list, we dynamically create it with None values
    # and get the correct dtype from the dtypes list declared outside of function
    for col in columns:
        if col not in df.columns:
            df = df.with_column(pl.lit(None, dtype=dtypes[col]).alias(col))
    df = df.select(columns)
    dfs.append(df)
    print(df.columns)
        
big_df = pl.concat(dfs)

In [ ]:
%%time
# big_df.to_pandas()
big_df

In [4]:
file1 = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_02_EXP_SELECT_CIK_CSV\35527\0000950152-04-008309.csv")
schema = pl.scan_csv(file1).schema

read_cols = list(set(schema.keys()).intersection(columns))

df1 = pl.read_csv(file1, columns=read_cols, dtypes=dtypes)

In [ ]:
# schema, 
df1.head(100)